### Programa de Automação de envio de relatório para gerência

In [ ]:
#Passo 1 Importar bibliotecas e bases de dados
import pandas as pd
import pathlib
import win32com.client as win32
emails = pd.read_excel(r'Bases de Dados/Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados/Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados/Vendas.xlsx')

#Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador
#incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')

#Fazer tabelas das lojas
dicionario_lojas = {}
for loja in lojas ['Loja']:
    dicionario_lojas[loja] =vendas.loc[vendas ['Loja'] == loja, :]

#definindo o dia
dia_indicador = vendas ['Data']. max ()

#Passo 3 - Salvar a planilha na pasta de backup
#Identificar se a pasta já existe
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')
arquivos_pasta_backup = caminho_backup.iterdir()

lista_nomes_backup = []
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for Loja in dicionario_lojas:
    if Loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / Loja
        nova_pasta.mkdir()
        
    #    Salvar dentro da pasta
    nome_arquivo = '{}_{}_{}.xlsx'.format (dia_indicador.month,dia_indicador.day,Loja)
    local_arquivo = caminho_backup / Loja / nome_arquivo
    dicionario_lojas[Loja].to_excel(local_arquivo)

#Passo 4 - Calcular o indicador para 1 loja
#Definição de metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeproduto_dia = 4
meta_qtdeproduto_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

#Passo 6 - Automatizar todas as lojas
for loja in dicionario_lojas:
    vendas_loja = dicionario_lojas[loja]
    
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador,:]

    #Faturamento
    faturamento_loja_ano = vendas_loja['Valor Final'].sum()
    faturamento_loja_dia = vendas_loja_dia['Valor Final'].sum()

    #Diversidade dos produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    #Ticket médio
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()

    #Ticket médio dia
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    

    #Passo 5-Enviar por e-mail para o gerente
    outlook = win32.Dispatch('outlook.application') 
    #s = win32.Dispatch("Mapi.Session")
    #s.Logon('outlook 2019')
    mail = outlook.CreateItem(0)
    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    mail.To = emails.loc[emails['Loja'] ==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia{dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    if faturamento_loja_dia >= meta_faturamento_dia :
        cor_fat_dia = "green"
    else: cor_fat_dia = "red"

    if faturamento_loja_ano >= meta_faturamento_ano :
        cor_fat_ano = "green"
    else: cor_fat_ano = "red"

    if qtde_produtos_dia >= meta_qtdeproduto_dia :
        cor_qtde_dia = "green"
    else: cor_qtde_dia = "red"

    if qtde_produtos_ano >= meta_qtdeproduto_ano :
        cor_qtde_ano = "green"
    else: cor_qtde_ano = "red" 

    if ticket_medio_dia >= meta_ticketmedio_dia :
        cor_ticket_dia = "green"
    else: cor_ticket_dia = "red"  

    if ticket_medio_ano >= meta_ticketmedio_ano :
        cor_ticket_ano = "green"
    else: cor_ticket_ano = "red"

    #mail.Body = "Texto do email"
    mail.HTMLBody = f'''
    <html>
    <p>Bom dia {nome},</p>

    <p> O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong> Loja {loja}</strong> foi:</p>

    <table>
        <tr>
            <th>Indicador</th>
            <th>Valor dia</th>
            <th>Meta dia</th>
            <th>Cenário dia</th>
        </tr>
            <td>Faturamento</td>
            <td style="text-align: center">R${faturamento_loja_dia:.2f}</td>
            <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
            <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
        <tr>
            <td>Diversidade de produtos</td>
            <td style="text-align: center">{qtde_produtos_dia}</td>
            <td style="text-align: center">{meta_qtdeproduto_dia}</td>
            <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
        </tr>
        <tr>
            <td>Ticket médio</td>
            <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
            <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
        </tr>
    </table>
    <br>
    <table>
        <tr>
            <th>Indicador</th>
            <th>Valor ano</th>
            <th>Meta ano</th>
            <th>Cenário ano</th>
        </tr>
            <td>Faturamento</td>
            <td style="text-align: center">R${faturamento_loja_ano:.2f}</td>
            <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
            <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
        <tr>
            <td>Diversidade de produtos</td>
            <td style="text-align: center">{qtde_produtos_ano}</td>
            <td style="text-align: center">{meta_qtdeproduto_ano}</td>
            <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
        </tr>
        <tr>
            <td>Ticket médio</td>
            <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
            <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
        </tr>
    </table>

    <p>Segue anexo planilha com todos os dados para mais detalhes</p>    

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Atenciosamente, Márcio</p>
    '''

    # Anexos (pode colocar quantos quiser):
    attachment = pathlib.Path.cwd()/f'{caminho_backup}'/f'{loja}'/f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))
    #print(attachment)
    mail.Send()
    print('Email da loja {} enviado'.format(loja))
    
#Passo 7 - Criar ranking para diretoria
faturamento_lojas = vendas.groupby('Loja')[['Loja','Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by = 'Valor Final', ascending = False)

nome_arquivo = '{}_{}_ranking anual.xlsx'.format (dia_indicador.month,dia_indicador.day)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas/{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data'] == dia_indicador]
faturamento_lojas_dia = vendas.groupby('Loja')[['Loja','Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by = 'Valor Final', ascending = False)

nome_arquivo = '{}_{}_ranking dia.xlsx'.format (dia_indicador.month,dia_indicador.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas/{}'.format(nome_arquivo))

#Passo 8 - Enviar e-mail para diretoria
outlook = win32.Dispatch('outlook.application')

nome = emails.loc[emails['Loja'] == 'Diretoria', 'Gerente'].values[0]
mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja'] == 'Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia{dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Presados, bom dia!

Melhor loja do dia em faturamento: Loja {faturamento_lojas_dia.index[0]} com faturamento de R${faturamento_lojas_dia.iloc[0,0]:.2f}
Pior loja do dia em faturamento: Loja {faturamento_lojas_dia.index[-1]} com faturamento de R${faturamento_lojas_dia.iloc[-1,0]:.2f}

Melhor loja do ano em faturamento: Loja {faturamento_lojas_ano.index[0]} com faturamento de R${faturamento_lojas_ano.iloc[0,0]:.2f}
Pior loja do ano em faturamento: Loja {faturamento_lojas_ano.index[-1]} com faturamento de R${faturamento_lojas_ano.iloc[-1,0]:.2f}

Seguem em anexo os rankings do dia e do ano de todas as lojas.
Qualquer dúvida, estou à disposição.
Atenciosamente,
Márcio.
'''

# Anexos (pode colocar quantos quiser):
Attachment = pathlib.Path.cwd()/f'{caminho_backup}'/f'{dia_indicador.month}_{dia_indicador.day}_ranking anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd()/f'{caminho_backup}'/f'{dia_indicador.month}_{dia_indicador.day}_ranking dia.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print('Email da Diretoria enviado')